In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

In [3]:
train = pd.read_csv("rossman_data/train.csv")

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [28]:
store = pd.read_csv("rossman_data/store.csv")

In [5]:
test = pd.read_csv("rossman_data/test.csv")

In [6]:
submission = pd.read_csv("rossman_data/sample_submission.csv")

In [7]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


In [9]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,1,0,0
1,2,3,4,2015-09-17,1,1,0,0
2,3,7,4,2015-09-17,1,1,0,0
3,4,8,4,2015-09-17,1,1,0,0
4,5,9,4,2015-09-17,1,1,0,0


Добавим в датафрейм номер месяца, недели, года и дня в качестве признаков.

In [10]:
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

train['Month'] = train.Date.map(lambda x: x.month)
train['DayOfYear'] = train.Date.map(lambda x: x.dayofyear)
train['Year'] = train.Date.map(lambda x: x.year)
train['Week'] = train.Date.map(lambda x: x.week)

test['Month'] = test.Date.map(lambda x: x.month)
test['DayOfYear'] = test.Date.map(lambda x: x.dayofyear)
test['Year'] = test.Date.map(lambda x: x.year)
test['Week'] = test.Date.map(lambda x: x.week)

Для каждого магазина посчитаем его средние и медианные продажи.

In [17]:
meanStoreSales = train.groupby("Store", as_index=False).Sales.mean().rename(columns = {"Sales":"MeanStoreSales"})
meanDayStoreSales = train.groupby(["Store", "DayOfWeek"], as_index=False).Sales.mean().rename(columns = {"Sales":"MeanDayStoreSales"})
medianStoreSales = train.groupby("Store", as_index=False).Sales.median().rename(columns = {"Sales":"MedianStoreSales"})
medianDayStoreSales = train.groupby(["Store", "DayOfWeek"], as_index=False).Sales.median().rename(columns = {"Sales":"MedianDayStoreSales"})

Добавим информацию о средних продажах магазинов в датафрейм.

In [18]:
train = pd.merge(train, meanStoreSales, on='Store')
train = pd.merge(train, meanDayStoreSales, on=['Store', 'DayOfWeek'])
train = pd.merge(train, medianStoreSales, on='Store')
train = pd.merge(train, medianDayStoreSales, on=['Store', 'DayOfWeek'])

test = pd.merge(test, meanStoreSales, on='Store')
test = pd.merge(test, meanDayStoreSales, on=['Store', 'DayOfWeek'])
test = pd.merge(test, medianStoreSales, on='Store')
test = pd.merge(test, medianDayStoreSales, on=['Store', 'DayOfWeek'])

Исправим(переведем в стандартные категориальные признаки) информацию о праздниках в различных магазинах.

In [22]:
train.SchoolHoliday = train.SchoolHoliday.apply(float)
test.SchoolHoliday = test.SchoolHoliday.apply(float)
train.StateHoliday = train.StateHoliday.apply(lambda x : str(x))
test.StateHoliday = test.StateHoliday.apply(lambda x : str(x))
test["Open"] = test["Open"].fillna(1)

stateHolidayMap = pd.Series([0, 1, 2, 3], index = ["0", "a", "b", "c"])
train.StateHoliday = train.StateHoliday.map(stateHolidayMap)
test.StateHoliday = test.StateHoliday.map(stateHolidayMap)

Исправим признаки о типе магазана, ассортимента и о интервали промоакций.

In [29]:
storeTypeMap = pd.Series([1, 2, 3, 4], index = ["a", "b", "c", "d"])
dictAssMap = pd.Series([1, 2, 3], index = ["a", "b", "c"])
promoIntMap = pd.Series([0, 1, 2, 3], index = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec'])

store.Assortment = store.Assortment.map(dictAssMap)
store.StoreType = store.StoreType.map(storeTypeMap)
store.PromoInterval = store.PromoInterval.map(promoIntMap)
store = store.fillna(0)

Добавим информацию о магазинах в датафрейм.

In [30]:
train = pd.merge(train, store, on="Store")
test = pd.merge(test, store, on="Store")

In [43]:
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

Определим функцию вычисления RMSPE.

In [64]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1.0 / (y[ind] ** 2)
    return w

def rmspe(yhat, y):
    w = ToWeight(yhat)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

In [65]:
def control(data, features, reg):
    controlCutDate = '2015-06-13 00:00:00'
    controlTrain = data[data.Date <= pd.Timestamp(controlCutDate)]
    controlTest = data[data.Date > pd.Timestamp(controlCutDate)]

    reg.fit(controlTrain[features], controlTrain.Sales)
    sales = reg.predict(controlTest[features])
    return rmspe(controlTest.Sales.values, sales)

Первый алгоритм регрессии - KNN resgression.

In [46]:
KNNFeatureList = ['Store', 'DayOfWeek', 'Promo', 'MedianStoreSales', 'Open', 'StateHoliday', 'CompetitionDistance', 'Assortment']

In [58]:
KNR = KNeighborsRegressor(n_neighbors = 13, leaf_size = 50)

print control(train, KNNFeatureList, KNR)

0.149174118231


In [22]:
KNR.fit(train[featureList], train.Sales)
predictSales = KNR.predict(test[featureList])
result = pd.DataFrame({'Id': test.Id.astype(int), 'Sales': predictSales}).set_index('Id').sort_index()
result.to_csv('knn.csv')

In [24]:
print KNNFeatureList

['Store', 'DayOfWeek', 'Promo', 'MedianStoreSales', 'Open', 'StateHoliday', 'CompetitionDistance', 'Assortment']


На локальном контроле для алгоритма KNN-регрессии получился результат 0.14917. В лидерборде получился результат 0.14605.

Второй алгоритм регрессии - градиентный буст.

In [48]:
GBRFeatureList = ['Store', 'DayOfWeek', 'Promo', 'MedianStoreSales', 'Open', 'StateHoliday', 'CompetitionDistance', 'Assortment']

In [66]:
GBR = GradientBoostingRegressor(loss = 'ls', n_estimators = 100, max_depth = 6)

print control(train, GBRFeatureList, GBR)

0.171455237012


In [51]:
GBR.fit(train[GBRFeatureList], train.Sales)
predictSales = GBR.predict(test[GBRFeatureList])
result = pd.DataFrame({'Id': test.Id.astype(int), 'Sales': predictSales}).set_index('Id').sort_index()
result.to_csv('gbr.csv')

Результат на контроле градиентного бустинга равен 0.17145. Результат в лидерборде 0.17789.

Третий алгоритм - случайный лес.

In [29]:
RFRFeatureList = ['Store', 'DayOfWeek', 'Promo', 'Open', 'MeanStoreSales',"Assortment"]

In [63]:
RFR = RandomForestRegressor(n_estimators=80, max_features = 'sqrt', n_jobs= -1)

print control(train, RFRFeatureList, RFR)

0.139341683401


In [31]:
RFR.fit(train[RFRFeatureList], train.Sales)
predictSales = RFR.predict(test[RFRFeatureList])
result = pd.DataFrame({'Id': test.Id.astype(int), 'Sales': predictSales}).set_index('Id').sort_index()
result.to_csv('rfr.csv')

На локальном контроле получился результат 0.1393. В лидерборде получился результат 0.14067.

In [71]:
d = {'Control':[0.1497, 0.1714, 0.13934], 'Leaderboard':[0.14605, 0.17789, 0.14067]}
df = pd.DataFrame(data = d, index = ['KNN regressor', 'Gradient Boostring regressorr', 'Random Forest regressor'])

Выведем оценки полученных алгоритмов в таблице.

In [70]:
df

,control,leaderboard
KNN regressor,0.14970,0.14605
Gradient Boostring regressorr,0.17140,0.17789
Random Forest regressor,0.13934,0.14067
